In [1]:
import requests
import json
from bs4 import BeautifulSoup
import config
import datetime
from datetime import timedelta

# Urls con api news y categorías con Beautiful Soup
El resultado es una lista "seleccionadas" con 5 notas por categoría

In [248]:
mydate = datetime.date(2018,9, 21) 
secciones = ['economia', 'deportes-2', 'sociedad', 'politica', 'teleshow/infoshow', 'america/mundo']
#también tiene entretenimiento, cual es la de internacionales, america + algo, mundo??

In [249]:
def obtener_urls(url):
    response = requests.get(url)
    google_soup = BeautifulSoup(response.text, 'html.parser' )
    urls = []
    for h3 in google_soup.find_all('h3'):
        for a in h3.find_all('a'):
            urls.append(a['href'].replace('/url?q=', ''))
    urls = [link for link in urls if seccion + '/' + año + '/' + mes + '/' +  dia + '/' in link]
    return urls

In [250]:
def obtener_nota(url):
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser' )
    titular = html_soup.find('h1').text
    try:
        autor= html_soup.find('a', {'class':'author-name'}).text
    except:
        autor = ""
    comentarios = bool(html_soup.find_all('h2', text = 'Comentarios'))
    cuerpo = ""
    for div in html_soup.find_all('div', {'id': 'article-body'}):
        for p in div.find_all('p'):
            cuerpo += p.text
    import re
    cuerpo = re.sub(r'Seguí leyendo.+', '', cuerpo).strip()
    cuerpo.replace('\xa0', ' ')
    nota = {'titulo': titular,
       'autor':autor,
       'cuerpo':cuerpo,
       'url':url,
       'fecha':fecha,
       'seccion': seccion,
       'abierta_comentarios': comentarios}
    return nota

In [246]:
%%time
notas = []
for dia in range(31):
    fecha = mydate.strftime("%d/%m/%y")
    año = '20' + mydate.strftime("%y")
    mes = mydate.strftime("%m")
    dia = mydate.strftime("%d")
    print('\n\n' + fecha)
    mydate += timedelta(days=1)
    for seccion in secciones:
        url = 'https://www.google.com.ar/search?ei=SlfLW6OEO8XI5gLvl5ngAg&q=*+site%3Ahttps%3A%2F%2Fwww.infobae.com%2F' + seccion + '%2F' + año + '%2F' + mes +'%2F' +dia+ '%2F&oq=*+site%3Ahttps%3A%2F%2Fwww.infobae.com%2F' + categ + '%2F' + año + '%2F' +mes+ '%2F' + dia + '%2F&gs_l=psy-ab.4...21974.22911.0.23438.4.4.0.0.0.0.260.513.2-2.2.0....0...1.1.64.psy-ab..2.0.0....0.xnbtghTKDls'
        urls = obtener_urls(url)
        print('\n' + seccion + ': ' + str(len(urls)))
        indice = 0
        while indice < 5:
            print(indice, end = ' ')
            if indice == len(urls):
                break
            notas.append(obtener_nota(urls[indice]))
            indice +=1


25/09/18

economia: 10
0 1 2 3 4 
deportes-2: 10
0 1 2 3 4 
sociedad: 8
0 1 2 3 4 
politica: 10
0 1 2 3 4 
teleshow/infoshow: 10
0 1 2 3 4 
america/mundo: 10
0 1 2 3 4 
26/09/18

economia: 10
0 1 2 3 4 
deportes-2: 10
0 1 2 3 4 
sociedad: 10
0 1 2 3 4 
politica: 10
0 1 2 3 4 
teleshow/infoshow: 10
0 1 2 3 4 
america/mundo: 10
0 1 2 3 4 Wall time: 1min 28s


In [253]:
pd.DataFrame(notas).to_csv('./data/infobae_notas.csv')

# Selenium para conseguir comments

In [161]:
import pandas as pd
df = pd.read_csv('./data/infobae_notas.csv', index_col = 0)

In [162]:
df.head(2)

abierta_comentarios autor  \
0                 True   NaN   
1                 True   NaN   

                                              cuerpo     fecha   seccion  \
0  Sorpresa: Luis Caputo, presentó este martes al...  25/09/18  economia   
1  El actual secretario de Política Económica, Gu...  25/09/18  economia   

                                              titulo  \
0              Luis Caputo renunció al Banco Central   
1  Guido Sandleris es el reemplazante de Caputo e...   

                                                 url  
0  https://www.infobae.com/economia/2018/09/25/lu...  
1  https://www.infobae.com/economia/2018/09/25/gu...

In [294]:
indice = 2
url = df.loc[indice].url
url

'https://www.infobae.com/economia/2018/09/25/argentina-registro-el-mayor-crecimiento-de-uber-en-todo-el-mundo/&sa=U&ved=0ahUKEwjs2cfPgJneAhXus1kKHWubC_sQFggfMAI&usg=AOvVaw3YOaiS6SMJkkQh2UWzJHnf'

In [358]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

path_to_extension = r'C:\Users\p_bag\OneDrive\Documentos\1.17.0_0'
path_to_extension_linux = r'/home/pablo/Documentos/1.17.0_0'
windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
linux = '/home/pablo/Descargas/chromedriver'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension_linux)


driver = webdriver.Chrome(linux, chrome_options=chrome_options)
driver.create_options()
driver.get(url)

/home/pablo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


In [359]:
#no es necesario cerrar el driver para cargar otra noticia

In [360]:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
iframe = driver.find_element_by_xpath("//*[@id='f0dmkLDVnJzT5r']/div[1]/span/iframe")
driver.switch_to.frame(iframe)
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))
    print ("Hay boton")
    button = True
except TimeoutException:
    print ("No hay boton")
    button = False
#Que pasa si la nota esta cerrada a comentarios

Hay boton


In [362]:
#creo que hay un tema con el tiempoq ue tarda en renderear la pagina
#element = driver.find_element_by_tag_name('button')
#element.location_once_scrolled_into_view
#driver.find_element_by_xpath("//button").click()

In [348]:
#driver.find_element_by_xpath("//*[contains(text(), 'Cargar')]").click()
#driver.find_element_by_xpath("//button").click()
#button = True
#ver que pasa que no lo hace de corrido?

In [349]:
while button:
    try:
        print('boton')
        element = driver.find_element_by_tag_name('button')
        element.location_once_scrolled_into_view
        time.sleep(2)
        driver.find_element_by_xpath("//button").click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))
        
    except: 
        button = False

intento


In [330]:
driver.switch_to.parent_frame()
driver.execute_script("window.scrollTo(0, 0)") 
driver.switch_to.frame(iframe)

In [315]:
#while(driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]")):
#    element = driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]")
#    element.location_once_scrolled_into_view
#    driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]").click()

In [324]:
button = True
while button:
    try:
        element = driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]")
        element.location_once_scrolled_into_view
        driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]").click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Mostrar')]")))

    except:
        button = False

In [145]:
#todos los comentarios con me gusta, horario y autores,etc
#todo = driver.find_elements_by_class_name('_3-8y')

#solo el texto
#todo = driver.find_elements_by_class_name('_3-8m')

In [146]:
#texto
#todo[2].find_element_by_class_name('_5mdd').text
#autor
#todo[2].find_element_by_class_name('UFICommentActorName').text

In [147]:
#me gusta _2vq9 fsm fwn fcg
#for x in todo[1].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span'):
#    print (x.text)

In [148]:
#fecha
#todo[1].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text

In [149]:
#me gusta (si es . no hay me gusta)
#todo[2].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text

In [303]:
#es respuesta?
todo = driver.find_elements_by_class_name('_5nz1')
len(todo)

56

In [305]:
%%time
comentarios = []
for item in todo:
    #es_respuesta = bool(item.find_elements_by_class_name('_3-8y'))
    print(item.find_element_by_class_name('_5mdd').text)
    comentarios.append({
        'autor': item.find_element_by_class_name('UFICommentActorName').text,
        'texto': item.find_element_by_class_name('_5mdd').text,
        'fecha': item.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text,
        'likes': item.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text,
        'respuestas' : len(item.find_elements_by_class_name('_3-8y')),
        'es_respuesta': False,
        'nota_id': indice,
        'nota_url': url
        })
    for respuesta in item.find_elements_by_class_name('_3-8y'):
        print(respuesta.find_element_by_class_name('_5mdd').text)
        comentarios.append({
            'autor': respuesta.find_element_by_class_name('UFICommentActorName').text,
            'texto': respuesta.find_element_by_class_name('_5mdd').text,
            'fecha': respuesta.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text,
            'likes': respuesta.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text,
            'respuestas' : len(respuesta.find_elements_by_class_name('_3-8y')),
            'es_respuesta': True,
            'nota_id': indice,
            'nota_url': url
             })

Que esperan para aceptar pagos con bitcoin??
El taxi no es monopolio si no todo lo contrario, Pepe el taxista no se queda con el 25% de lo que genera el resto de sus colegas.
Es un servicio regulado en base a una demanda limitada, la cual no se va a duplicar por duplicar los vehículos en servicio, prestado por licenciatarios que pagaron un canon por la explotación de tal servicio y que son, o dueños y trabajadores de su unidad, o pequeños empresarios que tienen trabajadores en relación de dependencia total hace años ya. Siquiera puede ser monotributista el chofer. El monopolio justamente es Uber, que plantea hacer trabajar a precios imp…Ver más
Leo ĺos comentarios y es mas que evidente que son ilegales los que opinan meten a todos en la misma bolsa y que no toman taxi el taxi tiene app targeta de credito ide tifivacion codigo QR placa y esta registrado en padron de sacta ente que lo regula seguro de transporte publico abilitacion y demas reqisitos y ademas se pagan impuestos la gente t

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"_5mdd"}
  (Session info: chrome=70.0.3538.67)
  (Driver info: chromedriver=2.42.591071 (0b695ff80972cc1a65a5cd643186d2ae582cd4ac),platform=Linux 4.15.0-34-generic x86_64)


In [293]:
import pandas as pd
pd.DataFrame(comentarios)

autor  es_respuesta  fecha likes  nota_id  \
0        Giuseppe Lescarafune         False  3 sem     ·        1   
1           Miguel Lafranconi         False  3 sem     ·        1   
2           Gonzalo Rodríguez         False  3 sem     ·        1   
3             Marian Iribarne          True  3 sem     ·        1   
4             Leonardo Planes         False  3 sem     ·        1   
5                Johny Malone         False  3 sem     ·        1   
6            Ricardo Gonzalez         False  3 sem     ·        1   
7     Alejandro Gubys Benetti         False  3 sem     ·        1   
8             Leonardo Planes          True  3 sem     ·        1   
9     Aispuru Alberto Horacio         False  3 sem     ·        1   
10         Eduardo P. Fabbian          True  3 sem     ·        1   
11                 Omar Zarza         False  3 sem     ·        1   
12      Pablo Javier Manrique         False  3 sem     1        1   
13               Domingo Cino         False  3 sem     ·        1   
14              Hector Ismael         False  3 sem     ·        1   
15     Diego Ignacio Peñaloza         False  3 sem     ·        1   
16            Leonardo Planes          True  3 sem     ·        1   
17     Diego Ignacio Peñaloza          True  3 sem     ·        1   
18                 José Saura         False  3 sem     1        1   
19                 Omar Zarza          True  3 sem     ·        1   
20                 Roly Lucho         False  3 sem     ·        1   
21               Cecilia Vaca         False  3 sem     ·        1   
22               Carlos Nadal         False  3 sem     ·        1   
23                 Omar Zarza          True  3 sem     ·        1   
24               Carlos Nadal          True  3 sem     ·        1   
25          Fernando Veleizan         False  3 sem     ·        1   
26   Roberto Enrique Gieschen         False  3 sem     2        1   
27            Javier Bruzzese          True  3 sem     ·        1   
28            Leonardo Planes          True  3 sem     ·        1   
29               Roberto Remo         False  3 sem     1        1   
..                        ...           ...    ...   ...      ...   
69               Carlos Diena         False  3 sem     1        1   
70            Eduardo Vanecek          True  3 sem     1        1   
71       Victoria Diaz Mendez          True  3 sem     ·        1   
72                Mariano Gds          True  3 sem     ·        1   
73             Carlos Sanchez         False  3 sem     4        1   
74            Cynthia Sanchez          True  3 sem     ·        1   
75                Mariano Gds          True  3 sem     ·        1   
76           Gustavo Seimandi         False  3 sem     1        1   
77            Edgardo Giudice         False  3 sem     6        1   
78             Ismael O Rossi         False      ·     ·        1   
79       Victoria Diaz Mendez         False  3 sem     2        1   
80           Alejandro Garcia          True  3 sem     ·        1   
81               Juan Falucho          True  3 sem     1        1   
82       Victoria Diaz Mendez          True  3 sem     ·        1   
83                  Nico Mars         False  3 sem     2        1   
84             Hernan Freschi          True  3 sem     2        1   
85               Juan Falucho          True  3 sem     ·        1   
86             Hernan Freschi          True  3 sem     ·        1   
87               Martin Lopez         False  3 sem     7        1   
88           Humberto Avesani          True  3 sem     ·        1   
89               Martin Lopez          True  3 sem     1        1   
90  Jose Luis Benitez Pallone          True  3 sem     ·        1   
91        Carlos Di Benedetto          True      ·     ·        1   
92               Martin Lopez          True  3 sem     ·        1   
93                Mariano Gds          True  3 sem     ·        1   
94             Eugenio Fresco         False  3 sem     2        1   
95       Luis Fernando

In [60]:
driver.quit()

In [61]:
#df.to_csv('./data/hugo-moyano-pedido-detencion-hijo.csv')

In [66]:
df

usuario                                            comment  \
0           LeoAlperi  Ya con lo que dijo ayer el Turco Assis que los...   
1    juanjorgesalzman  Lopez,Jaime,Baez,Zannini,Bousuo,De Vido,Manzan...   
2             Proteo1  Parece que no se usan los trolls para defender...   
3              mrojo3  este debería estar presi hace mucho y confisca...   
4            coco6344                             ¡Scioli no volvés más!   
5             mmurgia  Tiene también la fábrica de Felpudos jajajajaj...   
6            Vitolino  fue jefe de una asociación ilícita.  lo que es...   
7    juanjorgesalzman         Que raro Scioli sospechado de corrupcion….   
8           rarregui4  Este impresentable.en otro pais.ya hubiera cum...   
9          EricGaynor  Scioli hizo quebrar a todas las empresas que c...   
10           Saccardi  ...el motonauta quiso circular por la ancha av...   
11          cocoparra  que la justicia no deje de """visitar""" al cl...   
12            terviti  Es de esperar que salgan todas las irregularid...   
13      olages garcia   KARINA RABOLINI  YA DEVOLVIO LOS 800 MIL DOLA...   
14             aldina  @olages garcia  isculpe eran 2.millones de dól...   
15      olages garcia  @aldina ;  Uds, los macristas exageran todo pa...   
16     AlexOpenhaimer  Éste Scioli es el kirchnerista ? o el del Nuev...   
17           Saccardi  ...con el UPA, hace muchos años, aprendías a l...   
18             aldina  @Saccardi  no solo con las UPA,,,mentiroso y c...   
19               sii1  Aprieten las tuercas en la estafa de la Raboll...   
20           Saccardi              @sii1 ...otra cultora del poliamor...   
21     AlexOpenhaimer  @sii1 soltar la lengua.....le conviene a la le...   
22      olages garcia        @sii1 lo pregunto arriba, sin leerte. slds.   
23         alapipeta4  Por solo "2" puntos este delincuente casi es p...   
24           tucu2012                             De la que nos salvamos   
25           tucu2012                                Venezuela a un paso   
26     AlexOpenhaimer      @alapipeta4 Dios aprieta pero no ahorca !!!!!   
27           Saccardi  ...¿estos son los "desprolijos" que le gusta a...   
28             aldina  @Saccardi  bueh,ella con el marido que tiene,,...   
29             Jorako  Muchos creíamos que el felpudo era otro parril...   
..                ...                                                ...   
128           ChrisMB  Tarde o temprano tenía llegarle el turno al "a...   
129           gbarret                    No descorches. Aún no le llegó.   
130           samus26          El Peronio éste ya es "irregular" per sé.   
131   cundecambalache  MARCHE UN DESAFUERO EXPRESS EN 3...2...1...Y A...   
132   cundecambalache  ESTA ES LA RENOVACIÓN DE PERONIA SEÑORES.... L...   
133         Magda2010  Se apurò el fiscal Garganta despuès de la denu...   
134            xraiss                                   @Magda2010 CHORO   
135         Magda2010  @xraiss @Magda2010 Eso lo dirà la justicia,per...   
136       COCORICARDO                Y eso que tiene una sola mano.....    
137    armando_giusto                      Chiste mas original no tenés?   
138    armando_giusto            Al lado de estos Mauricio es Churchill.   
139          soyn2093  Ahora entienden por que entro 5to en una lista...   
140    armando_giusto          Scioli /Zanini y Anibaul...\nTe IMAGINAS?   
141          soyn2093  @armando_giusto Segun C5N ganaron por amplia d...   
142       mondonguito  Este no entra esposado. Entra con grilletes en...   
143          soyn2093  Yo te vote Macri pero ahora me arrepiento.. ah...   
144       critico2010  @soyn2093 somos dos... cuando veo lo del otro ...   
145     jnunezmendoza  pobre Yoli otro para Eceiza!!me parece que deb...   
146         Piolivera  ¿Podrá ser que no haya un solo peronista decente?   
147       critico2010  @Piolivera no pida imposibles. Está en sus gen...   
148         Femurroto          @Piolivera NO ES IMPOSIBLE !!!!